# Découverte du package Python-MIP
 - Correction
[Python-MIP](https://www.python-mip.com/) est un package python permettant la modélisation de programmes mathématiques, notamment des problèmes d'optimisation linéaire. C'est également une interface pour différents solveurs d'optimisation tels que CBC (solveur par défaut), HiGHS, etc.
Ce TP a pour objectif de se familiariser avec ce package.


## Installation 

Ce package est installé par défaut sur les ordinateurs de l'IUT. Autrement, il est possible d'installer le package `Python-mip` en exécutant dans une cellule de code l'instruction suivante :
```bash
pip install mip
```


## Utilisation

Pour utiliser le package `Python-mip`, il faut déjà l'inclure

In [ ]:
from mip import *

### Création du modèle

Pour définir un problème d'optimisation linéaire, on utilise la classe `Model`.

In [ ]:
m = Model()

### Ajout de variables dans le modèle

Pour déclarer une variable, on utilise la méthode `m.add_var`. Si l'on souhaite créer plusieurs variables, on peut créer un tableau contenant une variable par case.

**Remarques :**

- Il est possible de spécifier les bornes inférieure et supérieure pour chaque variable avec les paramètre `lb` et `ub`. **Attention :** par défaut, la borne inférieure est 0 (mettre `lb=float('-inf')` pour qu'une variable n'est pas de borne inférieure.
- Il est possible de donner un nom à une variable pour simplifier la lecture des fichiers au format lp.

In [ ]:
# Ajout de la variable y qui peut prendre n'importe quelle valeur (positive, négative ou nulle)
y = m.add_var(lb=float('-inf'))

# Pour créer 3 variables continues positives ou nulles
J = range(3)
x = [m.add_var() for j in J]

# On peut maintenant accéder aux variables avec x[0], x[1], x[2]

### Fonction objectif

Pour la fonction objective, on fixe la valeur de `m.objective` égale à une fonction linéaire que l'on optimise (`minimize` ou `maximize`).

In [ ]:
# Exemple, on souhaite maximiser 2 * x[0] + 3 * x[1] + x[2] + y
m.objective = maximize(2 * x[0] + 3 * x[1] - x[2] + y)

### Ajout de contraintes

Chaque contrainte est ajoutée dans le modèle en utilisant `m += `. Le signe est `<=`, `>=` ou `==`.

In [ ]:
# Ajout de la contrainte 2 x[0] + 3 x[1] <= 5
m += 2 * x[0] + 3 * x[1] <= 5

# Ajout de la contrainte x[0] + y = 2
m += x[0] + y == 2


### Optimisation et récupération de la solution

Une fois le modèle défini, on l'optimise avec `m.optimize()`. Cette fonction renvoie le status de l'optimisation :
- `OptimizationStatus.INFEASIBLE` si le problème n'est pas réalisable,
- `OptimizationStatus.UNBOUNDED` si le problème est non borné,
- `OptimizationStatus.OPTIMAL` autrement, c'est-à-dire s'il existe une solution optimale.

Si le problème est optimal, on récupère l'optimum avec `m.objective_value` et la solution optimale trouvée en utilisant la méthode `x` pour chaque variable.

In [ ]:
status = m.optimize()
if status == OptimizationStatus.OPTIMAL:
    optimum = m.objective_value
    solX = [ x[j].x for j in J]
    print(f"optimum = {optimum}")
    print(f"y = {y.x}")
    for j in J:
        print(f"x[{j}] = {solX[j]}")
else:
    print(f"status = {status}")


### Formulation générique

Jusqu'à présent, la fonction objectif et les contraintes sont définies de manière explicite. On peut faire une sommation implicite en utilisant la fonction `xsum`. Celle-ci permet de faire une somme à l'aide d'une boucle `for` pour créer la fonction objectif ou le membre de gauche d'une contrainte linéaire.
L'instruction est :
```python
xsum( ... for ...)
```
Par exemple, avec les variables précédentes, `x[0] + x[1] + x[2]` s'écrit également `xsum(x[j] for j in J)`.

De la même manière, `3 x[0] - x[1] + 2 x[2]` s'écrit `xsum(c[j] * x[j] for j in J)` avec `c = [3, -1, 2]`.

De plus, si les contraintes une même structure, il est possible de les ajouter à l'intérieur d'une boucle.

**Exemple :**
Supposons que l'on souhaite résoudre le problème
$$
\begin{align}
& \max x_0 + 2 x_1 + 3 x_2 + \dots + 10 x_9\\
& x_0 + x_1 + x_2 \leq 1\\
& x_1 + x_2 + x_3 \leq 1\\
& x_2 + x_3 + x_4 \leq 1\\
& \dots \\
& x_7 + x_8 + x_9 \leq 1\\
& x \geq \mathbf{0}
\end{align}
$$

Le problème peut donc se modélser de la manière suivante :


In [ ]:
def resolution():
    m = Model()
    m.verbose = 0 # pour supprimer les messages du solveur
    J = range(10)
    x = [ m.add_var() for j in J]
    m.objective = maximize(xsum( (j + 1) * x[j] for j in J))
    for i in range(8):
        # Ajout de la contrainte x[i] + x[i + 1] + x[i + 2] <= 1
        m += xsum(x[i + j] for j in range(3)) <= 1
    m.optimize()
    print(f"Optimum = {m.objective_value}")
    for j in J:
        print(f"x[{j}] = {x[j].x}")
resolution()


**Remarque :** Une bonne pratique consiste à créer le modèle à l'intérieur d'une fonction. Cela permet de recréer à chaque fois le modèle et évite des erreurs telles qu'ajouter une nouvelle contrainte au lieu de modifier une contrainte existante.

Pour plus d'information sur le package Python-MIP, vous pouvez consulter la [documentation](https://docs.python-mip.com/en/latest/index.html). 

## Exercice 1 : Fabrique de fibre optique


Une entreprise de fabrication de fibre optique doit décider de son plan de production pour la semaine suivante. Elle peut fabriquer trois types de fibre : 
 * Fibre Fluorée vendue à 35 &euro; le mètre
 * Fibre Chlorée vendue à 24 &euro; le mètre
 * Fibre Mélangée vendue à 39 &euro; le mètre

La fabrication de chaque type de fibre nécessite différentes ressources résumées dans le tableau ci-dessous.

| Ressources    | Fibre Fluorée  | Fibre Chlorée    | Fibre Mélangée    |
| ------------- | -------------: | ---------: | ---------: |
| Silice        | 10 kg/m        | 15 kg/m    | 15 kg/m    |
| Fluor         | 20 kg/m        | 0  kg/m    | 12 kg/m    |
| Chlore        | 0 kg/m         | 34 kg/m    | 18 kg/m    |
| Main-d'oeuvre | 3 h/m          | 4 h/m      | 6 h/m      |

L'entreprise possède une tonne de silice, 400kg de Fluor, 350kg de Clore. Par ailleurs, elle dispose d'une équipe de 7 ouvriers pouvant fabriquer les différents types de fibres optiques, chacun des ouvriers travaillant 35h par semaine. Elle souhaite maximiser son profit généré par la vente de ses différentes fibres (on suppose que l'entreprise peut vendre tout ce qu'elle produit. De plus, elle peut produire n'importe quelle quantité (même quelques microns de fibre optique).


### Question 1

Modéliser le problème avec `Python-mip` puis déterminer le plan de production optimal.


In [ ]:
##################
#   Correction   #
##################


def fabriqueDeFibre():

    # Création d'un modèle. 
    m = Model()
    m.verbose = 0 # Supprimer les log du solveur 
    
    F = m.add_var() # Quantité de fibre fluorée
    C = m.add_var() # Quantité de fibre chlorée
    M = m.add_var() # Quantité de fibre mélangée

    #Création de la fonction objective du modèle
    m.objective = maximize(35 * F + 24 * C  + 39 * M)

    #Ajout des contraintes
    m += 10 * F + 15 * C + 15 * M <= 1000 # Ressource silice
    m += 20 * F + 12 * M <= 400 # Ressource fluor
    m += 34 * C + 18 * M <= 350 # Ressource chlore
    m += 3 * F +  4 * C + 6 * M <= 7 * 35 # Main d'oeuvre

    # Résolution du problème d'optimisation linéaire m par le solveur GLPK
    status = m.optimize()

    # On vérifie que le problème est résolu à l'optimum
    assert status == OptimizationStatus.OPTIMAL


    print(f"Profit = {m.objective_value}€")
    print("Solution optimale :")

    print(f"\t Quantité de fibre fluorée = {round(F.x, 3)}m")
    print(f"\t Quantité de fibre chlorée = {round(C.x, 3)}m")
    print(f"\t Quantité de fibre mélangée = {round(M.x, 3)}m")


fabriqueDeFibre()

### Question 2 

Le service vente indique de d'après leurs estimations, il n'est pas possible de vendre plus de 15 mètres de fibre mélangée. De plus, il n'est pas possible de vendre plus de 20 mètres de fibre contenant du Chlore. Prendre en compte ces estimations dans le modèle.

In [ ]:
##################
#   Correction   #
##################


def fabriqueDeFibre():

    # Création d'un modèle. 
    m = Model()
    m.verbose = 0 # Supprimer les log du solveur 
    
    F = m.add_var() # Quantité de fibre fluorée
    C = m.add_var(ub = 20.) # Quantité de fibre chlorée 
    M = m.add_var(ub = 15.) # Quantité de fibre mélangée
    # Les contraintes de la question 2 sont ajoutées comme bornes supérieures pour les variables C et M

    #Création de la fonction objective du modèle
    m.objective = maximize(35 * F + 24 * C  + 39 * M)

    #Ajout des contraintes
    m += 10 * F + 15 * C + 15 * M <= 1000 # Ressource silice
    m += 20 * F + 12 * M <= 400 # Ressource fluor
    m += 34 * C + 18 * M <= 350 # Ressource chlore
    m += 3 * F +  4 * C + 6 * M <= 7 * 35 # Main d'oeuvre

    # Résolution du problème d'optimisation linéaire m par le solveur GLPK
    status = m.optimize()

    # On vérifie que le problème est résolu à l'optimum
    assert status == OptimizationStatus.OPTIMAL


    print(f"Profit = {round(m.objective_value, 2)}€")
    print("Solution optimale :")

    print(f"\t Quantité de fibre fluorée = {round(F.x, 3)}m")
    print(f"\t Quantité de fibre chlorée = {round(C.x, 3)}m")
    print(f"\t Quantité de fibre mélangée = {round(M.x, 3)}m")
    

fabriqueDeFibre()


### Question 3

L'entreprise peut embaucher une personne supplémentaire en intérim à 8 &euro; de l'heure (la personne peut travailler seulement quelques minutes et jusqu'à 35h). Est-ce intéressant ?

In [ ]:
##################
#   Correction   #
##################


def fabriqueDeFibre():

    # Création d'un modèle. 
    m = Model()
    m.verbose = 0 # Supprimer les log du solveur 
    
    F = m.add_var() # Quantité de fibre fluorée
    C = m.add_var(ub = 20.) # Quantité de fibre chlorée 
    M = m.add_var(ub = 15.) # Quantité de fibre mélangée
    I = m.add_var(ub = 35.) # Heures d'intérim
    

    #Création de la fonction objective du modèle
    m.objective = maximize(35 * F + 24 * C  + 39 * M - 8 * I)

    #Ajout des contraintes
    m += 10 * F + 15 * C + 15 * M <= 1000 # Ressource silice
    m += 20 * F + 12 * M <= 400 # Ressource fluor
    m += 34 * C + 18 * M <= 350 # Ressource chlore
    m += 3 * F +  4 * C + 6 * M <= 7 * 35 + I # Main d'oeuvre

    # Résolution du problème d'optimisation linéaire m par le solveur GLPK
    status = m.optimize()

    # On vérifie que le problème est résolu à l'optimum
    assert status == OptimizationStatus.OPTIMAL


    print(f"Profit = {round(m.objective_value, 2)}€")
    print("Solution optimale :")

    print(f"\t Quantité de fibre fluorée = {round(F.x, 3)}m")
    print(f"\t Quantité de fibre chlorée = {round(C.x, 3)}m")
    print(f"\t Quantité de fibre mélangée = {round(M.x, 3)}m")
    print(f"\t Intérim = {round(I.x, 3)}h")



fabriqueDeFibre()
# Il n'y a pas d'intérêt à embaucher un intérimaire

## Exercice 2

Un éleveur possède deux vaches de race charolaise et 5 vaches de race aveyronnaise. Pour nourrir son cheptel, il doit acheter un mélange de différentes céréales. Son objectif est d’acheter ces différentes céréales au coût minimum tout en s’assurant que chaque vache soit nourrie correctement. En effet, une vache charolaise a besoin de 40 mg de vitamine A, 50 mg de vitamine B et 100 mg de vitamine C par jour, alors qu’une vache aveyronnaise nécessite 70 mg de vitamine A, 60 mg de vitamine B et 30 mg de vitamine C. Il peut nourrir chaque vache avec différentes céréales : blé (5&euro; le kilogramme), maïs (4&euro; le kilogramme) et orge (7&euro; le kilogramme). Les teneurs en vitamines de ces différentes céréales sont données dans le tableau
suivant :

|     Céréales  | Vitamine A     | Vitamine B | Vitamine C |
| ------------- | -------------: | ---------: | ---------: |
| Blé           | 30 mg/kg       | 25 mg/kg   | 40 mg/kg   |
| Maïs          | 10 mg/kg       | 15 mg/kg   | 20 mg/kg   |
| Orge          | 50 mg/kg       | 45 mg/kg   | 35 mg/kg   |

### Question 1 

L’éleveur souhaite connaître les quantités des différentes céréales à acheter pour nourrir son troupeau au mois de mai. Modéliser son problème sous forme de PL. Indiquer la quantité (arrondie) de chaque céréales à acheter ainsi que le coût.

In [ ]:
##################
#   Correction   #
##################


import math # pour pouvoir utiliser m.ceil pour arrondir supérieurement

def achatCereales():

    m = Model()
    m.verbose = 0 # Supprimer les log du solveur 

    BA = m.add_var() # Quantité de blé pour les vaches aveyronnaises
    BC = m.add_var() # Quantité de blé pour les vaches charolaises
    OA = m.add_var() # Quantité de orge pour les vaches aveyronnaises
    OC = m.add_var() # Quantité de orge pour les vaches charolaises
    MA = m.add_var() # Quantité de maïs pour les vaches aveyronnaises
    MC = m.add_var() # Quantité de maïs pour les vaches charolaises

    #Création de la fonction objective du modèle
    m.objective = minimize(5 * BA + 5 * BC + 4 * MA + 4 * MC + 7 * OA +7 * OC)

    m += 30 * BA + 10 * MA + 50 * OA >= 70 * 5 * 31 # Vitamine A pour les vaches aveyronnaises
    m += 25 * BA + 15 * MA + 45 * OA >= 60 * 5 * 31 # Vitamine B pour les vaches aveyronnaises
    m += 40 * BA + 20 * MA + 35 * OA >= 30 * 5 * 31 # Vitamine C pour les vaches aveyronnaises
    m += 30 * BC + 10 * MC + 50 * OC >= 40 * 2 * 31 # Vitamine A pour les vaches charolaises
    m += 25 * BC + 15 * MC + 45 * OC >= 50 * 2 * 31 # Vitamine B pour les vaches charolaises
    m += 40 * BC + 20 * MC + 35 * OC >= 100 * 2 * 31 # Vitamine C pour les vaches charolaises


    # Résolution du problème d'optimisation linéaire
    status = m.optimize()

    # On vérifie que le problème est résolu à l'optimum
    assert status == OptimizationStatus.OPTIMAL

    print(f"Profit = {round(m.objective_value, 2)}€")
    print("Solution optimale :")

    print(f"\t Quantité de blé  (arrondie) = {math.ceil(BA.x + BC.x)}kg")
    print(f"\t Quantité de maïs (arrondie) = {math.ceil(MA.x + MC.x)}kg")
    print(f"\t Quantité d'orge  (arrondie) = {math.ceil(OA.x + OC.x)}kg")


achatCereales()

### Question 2 

Suite à un problème d’approvisionnement, l’éleveur ne peut pas acheter plus de 10 kilogrammes de blé. Par ailleurs, pour des questions de santé, l'orge ne peut  représenter plus du quart de la consommation de céréales d’une vache aveyronnaise. Modifier le PL en conséquence.

In [ ]:
##################
#   Correction   #
##################


def achatCereales():

    m = Model()
    m.verbose = 0 # Supprimer les log du solveur 

    BA = m.add_var() # Quantité de blé pour les vaches aveyronnaises
    BC = m.add_var() # Quantité de blé pour les vaches charolaises
    OA = m.add_var() # Quantité de orge pour les vaches aveyronnaises
    OC = m.add_var() # Quantité de orge pour les vaches charolaises
    MA = m.add_var() # Quantité de maïs pour les vaches aveyronnaises
    MC = m.add_var() # Quantité de maïs pour les vaches charolaises

    #Création de la fonction objective du modèle
    m.objective = minimize(5 * BA + 5 * BC + 4 * MA + 4 * MC + 7 * OA +7 * OC)

    m += 30 * BA + 10 * MA + 50 * OA >= 70 * 5 * 31 # Vitamine A pour les vaches aveyronnaises
    m += 25 * BA + 15 * MA + 45 * OA >= 60 * 5 * 31 # Vitamine B pour les vaches aveyronnaises
    m += 40 * BA + 20 * MA + 35 * OA >= 30 * 5 * 31 # Vitamine C pour les vaches aveyronnaises
    m += 30 * BC + 10 * MC + 50 * OC >= 40 * 2 * 31 # Vitamine A pour les vaches charolaises
    m += 25 * BC + 15 * MC + 45 * OC >= 50 * 2 * 31 # Vitamine B pour les vaches charolaises
    m += 40 * BC + 20 * MC + 35 * OC >= 100 * 2 * 31 # Vitamine C pour les vaches charolaises

    m += BA + BC <= 10
    m += 4 * OA <= OA + BA + MA

    # Résolution du problème d'optimisation linéaire
    status = m.optimize()

    # On vérifie que le problème est résolu à l'optimum
    assert status == OptimizationStatus.OPTIMAL

    print(f"Profit = {round(m.objective_value, 2)}€")
    print("Solution optimale :")

    print(f"\t Quantité de blé  (arrondie) = {math.ceil(BA.x + BC.x)}kg")
    print(f"\t Quantité de maïs (arrondie) = {math.ceil(MA.x + MC.x)}kg")
    print(f"\t Quantité d'orge  (arrondie) = {math.ceil(OA.x + OC.x)}kg")


achatCereales()